In [1]:
# Монтирование Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics easyocr roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 126.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 

In [3]:
import cv2
import time
import numpy as np
import pandas as pd
from ultralytics import YOLO
from IPython.display import HTML
from base64 import b64encode
import easyocr
from roboflow import Roboflow

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# Детекция и трекинг машин

Основные задачи:

 * Детекция и трекинг автомобилей :
    - Использование модели YOLOv8 для распознавания автомобилей.
    - Применение трекера BoT-SORT для отслеживания уникальных ID объектов.
    - Подсчет количества машин, пересекающих контрольную линию слева направо и
      справа налево.
 * Распознавание автомобильных номеров :
   - Обнаружение номерных табличек на автомобилях с помощью обученной модели
     YOLOv8.
   - Улучшение качества изображения номерных табличек с использованием методов
     обработки изображений.
   - Распознавание текста на номерных табличках с помощью библиотеки EasyOCR.
 * Визуализация результатов :
   - Создание тепловой карты плотности движения автомобилей.
   - Отображение bounding boxes с ID и классами объектов.
   - Сохранение обработанного видео с наложенными результатами.

## Данные
Для анализа предоставлены два видеофайла:

* Видео 1: [camera_uglich_shot.mp4](https://drive.google.com/file/d/1LSNU7ZAjZB1dnfrawOgSD_WLUJRz7BkM/view?usp=sharing) (дорога в Угличе, длительность 15 секунд).
* Видео 2: [camera_moscow.mp4](https://drive.google.com/file/d/1xqCXMk0p71EkpY9nbawiZwKJXUoG0Y4Q/view?usp=sharing) (дорога в Москве, длительность 17 секунд).

Оба видео содержат движение автомобилей по дорогам. Качество видео довольно среднее и детали, такие как автомобильные номера, практически не различимы. (ничего лучше найти и записать я не нашла)

### 1. Загрузка данных и инициализация

In [4]:
# Путь к основной папке
base_path = '/content/drive/MyDrive/Tracking'

# Путь к исходному видео
original_video_path = f'{base_path}/camera_uglich_shot.mp4'

# Путь к обработанному видео
tracked_video_path = f'{base_path}/tracked_video_uglich.mp4'

# Путь к файлу с результатами обнаруженных автомобильных номеров
csv_output_path = f'{base_path}/detected_plates_uglich.csv'

In [5]:
def display_video(file_path):
    mp4 = open(file_path, 'rb').read()
    data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
    return HTML(f"""
    <video width="640" height="320" controls>
        <source src="{data_url}" type="video/mp4">
    </video>
    """)

In [6]:
# Отображение оригинального видео
display_video(original_video_path)

Output hidden; open in https://colab.research.google.com to view.

### 2. Обнаружение и распознавание номерных табличек

* Номерные таблички обнаруживаются с помощью обученной модели YOLOv8. Так как  предобученная модель может распозновать только машины, то нужно было дообучить на данных с размеченными автомобильными номера (https://universe.roboflow.com/roboflow-universe-projects/license-plate-recognition-rxg4e), для распознования плашки.
* Изображение таблички обрабатывается для улучшения качества (преобразование в градации серого, пороговая обработка, размытие).
* Текст на табличке распознается с помощью библиотеки EasyOCR.

In [ ]:
rf = Roboflow(api_key="5pXFfA6Z3uRGVkNdeUMD")
project = rf.workspace("roboflow-universe-projects").project("license-plate-recognition-rxg4e")
version = project.version(6)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to License-Plate-Recognition-6 in yolov8:: 100%|██████████| 20262/20262 [00:02<00:00, 8061.84it/s]


In [ ]:
# Обучение модели для распознования автомобильных номеров
model_plates = YOLO('yolov8n.pt')
results = model_plates.train(data=f"{dataset.location}/data.yaml",
                             epochs=10,
                             imgsz=640,
                             device='cuda')

In [17]:
# Обнаружение номерной таблички
# Возвращает координаты bounding box
def detect_license_plate(image, model_plates):
    # Используем модель для предсказания
    results = model_plates(image)

    # Получаем координаты bounding box'а
    for result in results:
        boxes = result.boxes.data.tolist()
        for box in boxes:
            x1, y1, x2, y2, score, class_id = box
            # Предполагаем, что класс 0 - это номерная табличка
            if int(class_id) == 0:
                return int(x1), int(y1), int(x2), int(y2), score

    return None

# Обрезка и улучшение качества изображения номерной таблички
def preprocess_license_plate(image, bbox):
    x1, y1, x2, y2 = bbox

    # Обрезка изображения по координатам bounding box'а
    plate_image = image[y1:y2, x1:x2]

    # Преобразование в градации серого
    gray = cv2.cvtColor(plate_image, cv2.COLOR_BGR2GRAY)

    # Повышение контраста с помощью адаптивного порога
    _, thresholded = cv2.threshold(gray, 64, 255, cv2.THRESH_BINARY_INV)

    # Улучшение качества с помощью размытия
    blurred = cv2.GaussianBlur(thresholded, (3, 3), 0)

    return blurred

# Распознавание текста с помощью EasyOCR
def recognize_text(image):
    # Инициализация OCR
    reader = easyocr.Reader(['ru'], gpu=True)
    results = reader.readtext(image)

    # Обработка результатов
    for detection in results:
        bbox, text, score = detection
        # Очистка текста
        text = text.upper().replace(' ', '')
        return text, score

    return None, None

In [18]:
# Полный пайплайн обработки изображения
# 1. Обнаружение номерной таблички
# 2. Улучшение качества изображения
# 3. Распознавание текста
def pipline_image(image,
                  yolo_weights_path=f'{base_path}/runs/detect/train/weights/best.pt'):
    # Загрузка обученной модели YOLOv8
    model_plates = YOLO(yolo_weights_path)

    # Обнаружение номерной таблички
    bbox = detect_license_plate(image, model_plates)
    if bbox is None:
        print("\nНомерная табличка не найдена\n")
        return

    x1, y1, x2, y2, score = bbox
    print(f"\nНомерная табличка найдена с координатами: ({x1}, {y1}, {x2}, {y2}), уверенность: {score:.3f}\n")

    # Улучшение качества изображения
    processed_image = preprocess_license_plate(image, (x1, y1, x2, y2))

    # Сохранение обработанного изображения для проверки
    # cv2.imwrite('processed_plate.jpg', processed_image)
    # print("\nОбработанное изображение номерной таблички сохранено\n")

    # Шаг 3: Распознавание текста
    text, text_score = recognize_text(processed_image)
    if text is not None:
        print(f"\nРаспознанный текст: {text}, уверенность: {text_score:.3f}\n")
        return text, text_score
    else:
        print("\nТекст не распознан\n")
        return None

### 3. Детекция автомобилей :
* Используется предобученная модель YOLOv8 для распознавания автомобилей (класс car - 2).
* Для каждого автомобиля определяется bounding box, центр его координат и уникальный ID с помощью трекера BoT-SORT.

In [19]:
# Загрузка предобученной модели YOLOv8
model_cars = YOLO('yolov8n.pt')

In [20]:
tracker_config = 'botsort.yaml'
object_states = {}
crossing_count_left_to_right = 0
crossing_count_right_to_left = 0
detected_plates = []
heatmap = None

In [21]:
# Инициализация параметров видео
def initialize_video(original_video_path):
    cap = cv2.VideoCapture(original_video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    return cap, frame_width, frame_height, fps

# Инициализация видеозаписи
def initialize_output_video(tracked_video_path,
                            fps,
                            frame_width,
                            frame_height):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(tracked_video_path, fourcc, fps, (frame_width, frame_height))
    return out

### 4. Подсчет пересечений контрольной линии :
* На видео наносится горизонтальная линия, которая служит границей для подсчета автомобилей.
* При пересечении линии слева направо или справа налево увеличивается соответствующий счётчик.

In [22]:
# Создание линии подсчета
def define_count_line(frame_width, frame_height):
    y = int(frame_height - frame_height * 0.4)
    line_start = (0, y)
    line_end = (frame_width, y)
    return line_start, line_end

### 5. Создание тепловой карты

* Для каждого автомобиля на тепловой карте рисуется гауссиана в месте его центра.
* Тепловая карта накладывается на исходное видео для визуализации плотности движения.

In [23]:
# Инициализация тепловой карты
def initialize_heatmap(frame_height, frame_width):
    return np.zeros((frame_height, frame_width), dtype=np.float32)


# Рисование гауссианы в тепловой карте
def draw_gaussian_heatpoint(heatmap,
                            center,
                            radius=30,
                            strength=1.0):
    x, y = center
    size = int(radius * 2 + 1)
    gaussian = cv2.getGaussianKernel(size, radius / 3, ktype=cv2.CV_32F)
    kernel = np.outer(gaussian, gaussian.T)
    kernel = kernel / kernel.max() * strength

    height, width = heatmap.shape[:2]
    x0, y0 = max(0, x - radius), max(0, y - radius)
    x1, y1 = min(width, x + radius + 1), min(height, y + radius + 1)

    kernel_x0 = max(0, radius - x)
    kernel_y0 = max(0, radius - y)
    kernel_x1 = kernel_x0 + x1 - x0
    kernel_y1 = kernel_y0 + y1 - y0

    if x0 >= width or y0 >= height or x1 <= 0 or y1 <= 0:
        return

    heatmap[y0:y1, x0:x1] += kernel[kernel_y0:kernel_y1, kernel_x0:kernel_x1]

# Наложение тепловой карты
def apply_heatmap_overlay(frame, heatmap):
    heatmap_normalized = cv2.normalize(heatmap,
                                       None,
                                       0,
                                       255,
                                       cv2.NORM_MINMAX)
    heatmap_colored = cv2.applyColorMap(heatmap_normalized.astype(np.uint8),
                                        cv2.COLORMAP_JET)

    if heatmap_colored.shape[:2] != frame.shape[:2]:
        heatmap_colored = cv2.resize(heatmap_colored,
                                     (frame.shape[1], frame.shape[0]))

    alpha = 0.3
    return cv2.addWeighted(frame,
                           1 - alpha,
                           heatmap_colored,
                           alpha,
                           0)

In [24]:
# Обработка треков объектов
def process_tracks(model_cars,
                   frame,
                   line_start,
                   line_end,
                   pipeline_image):
    global crossing_count_left_to_right, crossing_count_right_to_left, object_states, heatmap

    results = model_cars.track(frame,
                               persist=True,
                               tracker=tracker_config,
                               classes=[2])

    for result in results:
        boxes = result.boxes.cpu().numpy()
        track_ids = boxes.id.astype(int) if boxes.id is not None else []

        for box, track_id in zip(boxes, track_ids):
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

            # Обновление тепловой карты
            draw_gaussian_heatpoint(heatmap, (cx, cy), strength=10)

            # Инициализация состояния объекта
            if track_id not in object_states:
                object_states[track_id] = {
                    'above_line': cy < line_start[1],
                    'counted_left_to_right': False,
                    'counted_right_to_left': False
                }

            prev_above_line = object_states[track_id]['above_line']
            current_above_line = cy < line_start[1]

            # Лево -> Право
            if prev_above_line and not current_above_line and not object_states[track_id]['counted_left_to_right']:
                crossing_count_left_to_right += 1
                object_states[track_id]['counted_left_to_right'] = True

            # Право -> Лево
            if not prev_above_line and current_above_line and not object_states[track_id]['counted_right_to_left']:
                crossing_count_right_to_left += 1
                object_states[track_id]['counted_right_to_left'] = True

            object_states[track_id]['above_line'] = current_above_line

            # Распознавание номеров
            cropped_plate = frame[y1:y2, x1:x2]
            ocr_results = pipeline_image(cropped_plate)

            plate_text = ""
            if ocr_results:
                text, confidence = ocr_results
                if confidence > 0.5:
                    plate_text = text

            # Логирование распознанного номера
            if plate_text:
                timestamp = time.strftime('%H:%M:%S', time.gmtime(cv2.CAP_PROP_POS_MSEC))
                detected_plates.append({
                    'timestamp': timestamp,
                    'track_id': track_id,
                    'plate': plate_text
                })

            # Отрисовка
            color = (0, 0, 255) if (object_states[track_id]['counted_left_to_right'] or
                                    object_states[track_id]['counted_right_to_left']) else (255, 0, 0)
            cv2.rectangle(frame,
                          (x1, y1),
                          (x2, y2),
                          color,
                          2)
            cv2.putText(frame,
                        f'ID: {track_id}',
                        (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.9,
                        color,
                        2)
            cv2.circle(frame,
                       (cx, cy),
                       5,
                       color,
                       -1)

            if plate_text:
                cv2.putText(frame,
                            f'Plate: {plate_text}',
                            (x1, y1 - 30),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.7,
                            (31, 43, 61),
                            2)

    return frame

### 6. Сохранение результатов
* Обработанное видео сохраняется с bounding boxes, тепловой картой и счетчиками.
* Результаты распознавания номерных табличек сохраняются в CSV-файл.

In [25]:
# Вывод счетчиков движения
def draw_crossing_counters(frame,
                           left_to_right,
                           right_to_left):
    cv2.putText(frame,
                f'Right to Left: {right_to_left}',
                (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (10, 38, 165),
                2)
    cv2.putText(frame,
                f'Left to Right: {left_to_right}',
                (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (10, 38, 165),
                2)
    return frame


# Сохранение результатов
def save_results(detected_plates,
                 base_path, tracked_video_path,
                 csv_output_path=f'{base_path}/detected_plates.csv'):
    df = pd.DataFrame(detected_plates)
    df.to_csv(csv_output_path, index=False)
    print(f'Логи распознанных номеров сохранены по пути: {csv_output_path}')
    print(f'Видео с тепловой картой и номерами сохранено по пути: {tracked_video_path}')

In [26]:
# Основной цикл обработки кадров
def run_tracking_pipeline(original_video_path,
                          tracked_video_path,
                          base_path,
                          model_cars,
                          pipeline_image,
                          csv_output_path):
    global heatmap

    cap, frame_width, frame_height, fps = initialize_video(original_video_path)
    line_start, line_end = define_count_line(frame_width, frame_height)
    out = initialize_output_video(tracked_video_path,
                                  fps,
                                  frame_width,
                                  frame_height)
    heatmap = initialize_heatmap(frame_height, frame_width)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Рисуем линию подсчёта
        cv2.line(frame, line_start, line_end, (255, 0, 139), 2)

        # Обрабатываем треки
        processed_frame = process_tracks(model_cars,
                                         frame,
                                         line_start,
                                         line_end,
                                         pipeline_image)

        # Накладываем тепловую карту
        final_frame = apply_heatmap_overlay(processed_frame, heatmap)

        # Отображаем счётчики
        final_frame = draw_crossing_counters(final_frame,
                                             crossing_count_left_to_right,
                                             crossing_count_right_to_left)

        # Запись кадра
        out.write(final_frame)

    # Освобождение ресурсов
    cap.release()
    out.release()

    # Сохраняем CSV
    save_results(detected_plates,
                 base_path,
                 tracked_video_path,
                 csv_output_path)

### 7. Тестирование

In [27]:
run_tracking_pipeline(
    original_video_path=original_video_path,
    tracked_video_path=tracked_video_path,
    base_path=base_path,
    model_cars=model_cars,
    pipeline_image=pipline_image,  # OCR-модель
    csv_output_path=csv_output_path
)

requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 0.7s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


0: 288x640 1 car, 51.1ms
Speed: 15.9ms preprocess, 51.1ms inference, 404.8ms postprocess per image at shape (1, 3, 288, 640)

0: 352x640 (no detections), 36.0ms
Speed: 1.5ms preprocess, 36.0ms inference, 0.9ms postprocess per image at shape (1, 3, 352, 640)

Номерная табличка не найдена


0: 288x640 1 car, 7.2ms
Speed: 2.9ms preprocess, 7.2ms inference, 2.0ms postprocess per image at shape (1, 3, 288, 640)

0: 352x640 (no detections), 9.3ms
Speed: 1.7ms preprocess, 9.3ms inference, 0.8ms postprocess per image at shape (1, 3, 352, 640)

Номерная табличка не найдена


0: 288x640 1 car, 7.6ms
Speed: 2.0ms preprocess, 7.6ms inference, 1.9ms postprocess per image at shape (1, 3, 288, 640)

0: 352x640 (no detections), 13.5ms
Speed: 2.1ms preprocess, 13.5ms inference, 0.8ms post


Номерная табличка найдена с координатами: (1, 1, 184, 103), уверенность: 0.332

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Выходные данные были обрезаны до нескольких последних строк (5000).

0: 320x640 1 License_Plate, 7.3ms
Speed: 6.1ms preprocess, 7.3ms inference, 1.4ms postprocess per image at shape (1, 3, 320, 640)

Номерная табличка найдена с координатами: (1, 5, 315, 149), уверенность: 0.813


Текст не распознан


0: 288x640 2 cars, 9.9ms
Speed: 2.9ms preprocess, 9.9ms inference, 2.4ms postprocess per image at shape (1, 3, 288, 640)

0: 416x640 1 License_Plate, 7.0ms
Speed: 1.6ms preprocess, 7.0ms inference, 1.4ms postprocess per image at shape (1, 3, 416, 640)

Номерная табличка найдена с координатами: (62, 43, 86, 51), уверенность: 0.590


Текст не распознан


0: 320x640 1 License_Plate, 16.1ms
Speed: 2.0ms preprocess, 16.1ms inference, 2.9ms postprocess per image at shape (1, 3, 320, 640)

Номерная табличка найдена с координатами: (2, 6, 314, 147), уверенность: 0.847


Текст не распознан


0: 288x640 2 cars, 10.1ms
Speed: 4.7ms preprocess, 10.1ms inference, 2.2ms postprocess per image at shape (1

In [32]:
# Отображение видео с трекингом
display_video(tracked_video_path)

Output hidden; open in https://colab.research.google.com to view.

In [29]:
# Путь к исходному видео
original_video_path = f'{base_path}/camera_moscow.mp4'

# Путь к обработанному видео
tracked_video_path = f'{base_path}/tracked_video_moscow.mp4'

# Путь к файлу с результатами обнаруженных автомобильных номеров
csv_output_path = f'{base_path}/detected_plates_moscow.csv'

In [30]:
run_tracking_pipeline(
    original_video_path=original_video_path,
    tracked_video_path=tracked_video_path,
    base_path=base_path,
    model_cars=model_cars,
    pipeline_image=pipline_image,  # OCR-модель
    csv_output_path=csv_output_path
)

Выходные данные были обрезаны до нескольких последних строк (5000).

0: 448x640 (no detections), 6.7ms
Speed: 1.5ms preprocess, 6.7ms inference, 0.7ms postprocess per image at shape (1, 3, 448, 640)

Номерная табличка не найдена


0: 480x640 (no detections), 6.8ms
Speed: 1.7ms preprocess, 6.8ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

Номерная табличка не найдена


0: 448x640 (no detections), 6.7ms
Speed: 1.6ms preprocess, 6.7ms inference, 0.6ms postprocess per image at shape (1, 3, 448, 640)

Номерная табличка не найдена


0: 544x640 (no detections), 7.8ms
Speed: 1.9ms preprocess, 7.8ms inference, 0.7ms postprocess per image at shape (1, 3, 544, 640)

Номерная табличка не найдена


0: 640x512 (no detections), 8.9ms
Speed: 2.5ms preprocess, 8.9ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 512)

Номерная табличка не найдена


0: 512x640 (no detections), 11.6ms
Speed: 2.6ms preprocess, 11.6ms inference, 1.1ms postprocess per image at shape (1, 3,

In [33]:
# Отображение видео с трекингом
display_video(tracked_video_path)

Output hidden; open in https://colab.research.google.com to view.

## Результаты

* [Видео 1: Дорога в Угличе](https://drive.google.com/file/d/1J_caJ6KrH6_tYpekPuTmqRTv7Nvuf3BN/view?usp=sharing)

  * YOLOv8 успешно распознавала большинство автомобилей. Однако наблюдались проблемы с "скачущими" bounding boxes и идентификацией, особенно для машины с дополнительным грузом, которая была ошибочно определена как две машины.

  * Подсчет пересечений контрольной линии сработал корректно для большинства автомобилей.

  * Номерные таблички не были распознаны ни для одного автомобиля. Это связано с тем, что качество видео недостаточно для четкого выделения текста на табличках. (https://drive.google.com/file/d/1NjjX59p_fUuweM3VbHYNmk3LscK1F0b6/view?usp=sharing)

  * Тепловая карта неплохо визуализировала плотность движения автомобилей.

* [Видео 2: Дорога в Москве](https://drive.google.com/file/d/12pDzpEyCmBk11ec9l4xLDAkyoAzsCGZn/view?usp=sharing)

  * YOLOv8 пропустила много автомобилей, особенно на участках с низкой освещенностью. Наблюдались проблемы с "скачущими" bounding boxes и идентификацией.

  * Многие машины при пересечении контрольной линии не окрашивались в другой цвет, что указывает на ошибки в работе трекера.

  * Как и в первом видео, номерные таблички не были распознаны ни для одного автомобиля. Человеческим взглядом также невозможно разглядеть номера на видео из-за низкого качества. (https://drive.google.com/file/d/1icXahcdaip8JwrxXLYDgRF_Ad50M86Hm/view?usp=sharing)

  * Тепловая карта показала общую активность движения, но из-за пропусков в детекции автомобилей карта менее информативна.



### Выводы

* YOLOv8 показала приемлемые результаты на видео с более лучшим качеством (Углич), но на видео с низким качеством (Москва) производительность значительно ухудшилась. Взять модель YOLO11 не было возможности из-за ограниченных ресурсов.

* Проблемы со "скачущими" bounding boxes и пропусками автомобилей требуют дальнейшей настройки модели и трекера.

* Распознавание номеров оказалось невозможным из-за низкого качества видео.
Для успешного распознавания номеров требуется видео с более высоким разрешением и четкостью.

* Тепловая карта успешно визуализировала плотность движения, но ее точность зависит от качества детекции автомобилей.

### Что можно улучшить

* Использовать более качественные видео для анализа. Но, к сожалнеию, все камеры на geocam.ru либо не очень высокого качества, либо не работают.

* Дополнительно настроить модель YOLOv8 и трекер BoT-SORT для улучшения стабильности детекции.

* Применять другие методы повышения качества изображения перед распознаванием номерных табличек.